# Hackathon

## Input

In [1]:
import pandas as pd

In [2]:
def getProblemData(path):
    dict_problem = dict()
    dict_flights = dict()
    dict_arc = dict()
    
    dict_flights["v"] = []
    dict_flights["c"] = []
    
    dict_arc["a"] = []
    dict_arc["o"] = []
    dict_arc["d"] = []
    dict_arc["t"] = []
    dict_arc["n"] = []

    with open(path, "r") as infile:
        for i, line in enumerate(infile):
            if i==0:

                list_element = line.split(" ")
                
                dict_problem["V"] = int(list_element[1])
                dict_problem["A"] = int(list_element[3])
                dict_problem["P"] = int(list_element[5])
                dict_problem["B"] = int(list_element[7])
                dict_problem["K"] = int(list_element[9])
                dict_problem["G"] = int(list_element[11])
                
            else:
                
                
                
                if line.find("v") != -1:
                    list_element = line.split(" ")
                    dict_flights["v"] += [int(list_element[1]) - 1] 
                    dict_flights["c"] += [[int(el) for el in list_element[3::]]]

                elif line.find("a") != -1:
                    list_element = line.split(" ")
                    dict_arc["a"] += [int(list_element[1]) - 1]
                    dict_arc["o"] += [int(list_element[3]) - 1]
                    dict_arc["d"] += [int(list_element[5]) - 1]
                    dict_arc["t"] += [int(list_element[7])]
                    dict_arc["n"] += [int(list_element[9])]
                    
    df_flights = pd.DataFrame(dict_flights, index = dict_flights["v"])
    df_arc = pd.DataFrame(dict_arc, index = dict_arc["a"])
    
    return dict_problem, df_flights, df_arc

In [69]:
instance = "S"

In [70]:
dict_problem, df_flight, df_arc = getProblemData("instance_"+instance+".in")

In [71]:
df_flight.head()

,c,v
0,"[12, 15, 10]",0
1,"[8, 9, 10]",1
2,"[5, 6, 7]",2
3,"[2, 3, 4]",3
4,"[4, 5, 6]",4


In [72]:
df_arc.head()

,a,d,n,o,t
0,0,2,5,0,1
1,1,3,1,0,1
2,2,2,4,1,1
3,3,3,0,1,0
4,4,5,3,4,1


In [73]:
P = dict_problem['P']

for p in range(P):
    df_flight["C_P"+str(p)] = df_flight.apply(lambda x : x.c[p], axis=1)


In [74]:
print dict_problem
df_flight

{'A': 8, 'B': 5000, 'G': 1000, 'K': 5, 'P': 3, 'V': 7}


,c,v,C_P0,C_P1,C_P2
0,"[12, 15, 10]",0,12,15,10
1,"[8, 9, 10]",1,8,9,10
2,"[5, 6, 7]",2,5,6,7
3,"[2, 3, 4]",3,2,3,4
4,"[4, 5, 6]",4,4,5,6
5,"[7, 8, 9]",5,7,8,9
6,"[1, 3, 2]",6,1,3,2


# Graph

In [75]:
class Graph:

    def __init__(self):
        self.c = 0 #counter seen
        self.A = int(dict_problem["A"])
        self.V = int(dict_problem["V"])
        self.am = []
        self.seen = [False] * self.V # visité
        self.am = [] * self.A # list of boolean a \in Am 
        self.ta = [[] * self.V]*self.V #  ta cost (in days)
        
        self.depth = [0]*self.V
        
        self.adj = [] # adjacence
        for i in range(self.V):
            self.adj += [[]]

        for index, row in df_arc.iterrows():
            self.adj[row["o"]] += [row["d"]]
        
        
    
    def create():
        print "creer les données"
        

    def dfs_recursive(self, graph, node, seen):
        """DFS, detect connected component, recursive implementation
        :param graph: directed graph in listlist or listdict format
        :param int node: to start graph exploration
        :param boolean-table seen: will be set true for the connected component
              containing node.
        :complexity: `O(|V|+|E|)`
        """
        #seen = [False]*self.V
        seen[node] = True # visité
        a = 0
        
        to_visit = []
        for i in graph[node]:
            if not seen[i]:
                to_visit.append(i)
        
        for neighbor in to_visit: 
                a += 1 + self.dfs_recursive(graph, neighbor, seen)
        return a / len(to_visit) if a > 0 else 0

    
    
    def select_next_node(self, p, graph, node, seen, df_flight, df_arc):
        neighbors = self.adj[node]
        print "N : ", node, " : ",neighbors
        
        if len(neighbors) > 0:
            nodes = []
            
            df_tmp = df_arc.merge(df_flight, how='inner', left_on='d', right_on='v')
            df_tmp = df_tmp[df_tmp.o == node]
            if df_tmp.shape[0] > 0:
                seen2 = list(self.seen)
                df_tmp['depth'] = df_tmp.apply(lambda x : g.dfs_recursive(self.adj, x.d, list(self.seen)), axis=1) 
                df_tmp['crit'] = df_tmp.apply(lambda x : (1.0*x.depth)/(x.n * x['C_P'+str(p)]+1), axis=1) #cost                               
                #print df_tmp
                next_node = int(df_tmp.sort_values(by="crit", ascending=False).head(1).d) # selection du min(x) 
                print "next : ", next_node
                return next_node
        
        return -1


    #graph = self.adj
    def dfs_recursive_red(self, p, graph, node, seen, path):
        """DFS, detect connected component, recursive implementation
        :param graph: directed graph in listlist or listdict format
        :param int node: to start graph exploration
        :param boolean-table seen: will be set true for the connected component
              containing node.
        :complexity: `O(|V|+|E|)`
        """
        self.seen[node] = True # visité
        #for neighbor in graph[node]:
        neighbor = self.select_next_node(p, graph, node, self.seen, df_flight, df_arc) # TO test : prendre le noeud dont cvp*ta est minimal et non visité au lieu du for
        print neighbor
        self.c += 1 if self.seen[neighbor] else 0
        print "c : ", self.c
        if neighbor >= 0 and self.c <= P/3:
            if (self.seen[neighbor] and len(g.adj[neighbor]) > 0) or not self.seen[neighbor]:
                path.append(neighbor) # add neighor dans le path 
                #if not self.seen[neighbor] and self.c <= P/3 :
                self.dfs_recursive_red(p, graph, neighbor, seen, path) # to up
        
    
    
    def select_first_node(self, p, df_arc, df_flight):
        """
        :nodes list de noeud
        :p num avion
        :df df_flight
        """
        nodes = []
        for i in range(len(self.seen)):
            if not self.seen[i]:
                nodes.append(i)
                
        print "p : ", p, " : ", nodes
        
        df_tmp = df_arc.merge(df_flight, how='inner', left_on='o', right_on='v')
        
        #print self.dfs_recursive( self.adj, 0, self.seen)
        
        df_tmp = df_tmp[df_tmp.o.isin(nodes)]  # not seen
        if df_tmp.shape[0] > 0:
            df_tmp2 = df_tmp[df_tmp.t == 0] #not_AM (if possible)
            if df_tmp2.shape[0] > 0:
                df_tmp = df_tmp2

        seen2 = list(self.seen)
        df_tmp['depth'] = df_tmp.apply(lambda x : self.dfs_recursive(g.adj, x.v, list(self.seen)) , axis=1) 
        #df_tmp['depth'] = df_tmp.apply(lambda x : len(self.adj[x.v]) , axis=1) 
        #print df_tmp        
                
        df_tmp['crit'] = df_tmp.apply(lambda x : (1.0*x.depth)/(x["C_P"+str(p)]+1), axis=1)                                
        df_tmp = df_tmp.sort_values(by=["crit"], ascending=False)
        print "First node : ", int(df_tmp.head(1).v)
        return int(df_tmp.head(1).v)
        
        
 

# Test greedy _red

In [76]:
def greedy_red(P, g, df_flight, df_arc):
    """
    :g graph
    :P nb avions
    """
    rotation = [[]]*P
    for p in range(P):
        path = []
        node = g.select_first_node(p, df_arc, df_flight) # To TEST : select le premier noeud parmi les dispos, avec c_vp minimal
        path = [node] # init path
        g.dfs_recursive_red(p, g.adj, node, g.seen, path) # TO TEST
        print "path : ", path
        rotation[p] = path
    return rotation
    
   

In [77]:
%%time
g = Graph()
rot = greedy_red(P, g, df_flight, df_arc)

p :  0  :  [0, 1, 2, 3, 4, 5, 6]
First node :  1
N :  1  :  [2, 3]
next :  3
3
c :  0
N :  3  :  [5, 6]
next :  5
5
c :  0
N :  5  :  []
-1
c :  0
path :  [1, 3, 5]
p :  1  :  [0, 2, 4, 6]
First node :  4
N :  4  :  [5, 6]
next :  5
5
c :  1
path :  [4]
p :  2  :  [0, 2, 6]
First node :  0
N :  0  :  [2, 3]
next :  3
3
c :  2
path :  [0]
Wall time: 369 ms


In [78]:
print rot

[[1, 3, 5], [4], [0]]


In [19]:
def init_solution():

    # List of rotations
    rotations = []
    for i in range(dict_problem["P"]):
        rotations += [[]]
    print("Rotation :"+str(len(rotations)))
    
    # Flights and their assigned rotations
    flights_rotations = []
    for i in range(dict_problem["V"]):
        flights_rotations += [[]]
    print("Flights :"+str(len(flights_rotations)))
    
    return rotations, flights_rotations

In [20]:
def greedySolution():

    for r in range(dict_problem["P"]):
        for f in range(dict_problem["V"]):
            #print("f: " + str(f))
            if len(flights_rotations[f]) > 0:
                continue
            else:
                next_flight = f

                while next_flight is not None:
                    current_flight = next_flight
                    flights_rotations[current_flight] += [r]
                    rotations[r] += [current_flight]

                    next_flight = None
                    for neighb in graph.adj[current_flight]:
                        if len(flights_rotations[neighb]) == 0:
                            next_flight = neighb
                            break # we found the next flight
                break # finish the rotation
    return

In [21]:
graph = Graph()

rotations, flights_rotations = init_solution()

greedySolution()
#rotations = greedy_red(P, g, df_flight, df_arc)

Rotation :3
Flights :7


## Cost function

In [22]:
assignment_cost = 0
for fr in flights_rotations:
    assignment_cost += dict_problem["B"] * abs(len(fr) - 1)

In [23]:
assignment_cost

0

In [24]:
V_maintenance = [] * P

In [25]:
ope_cost = 0
for r in rotations:
    for f in r:
        ope_cost += graph.cost[f]

AttributeError: Graph instance has no attribute 'cost'

In [ ]:
maintenance_cost = 0

In [35]:
maintenance_cost += dict_problem["G"] * sum(V_maintenance)

## Output

In [79]:
OUTPUT_FILE_NAME = "output_"+instance+"_after.dat"
rotations = rot

In [80]:
with open(OUTPUT_FILE_NAME, "w") as output_file:
    for r_i, r in enumerate(rotations):
        output_file.write("p " + str(r_i+1) + " a")
        for flight_num in r:
            output_file.write(" " + str(flight_num+1))
        output_file.write("\n")
        
